### čtvrtá lekce

2. `pandas` II.,
    - file io, encoding, chybějící data,
    - kombinace datasetů, concat, append, merge, join,
    - časové řady, možná?
    - eval() a query() jako high-perf operace.

### třetí lekce

2. `pandas` I.,
    - Úvod do frameworku, proč je dobré jej ovládat, kde s ním pracovat?
    - jaké jsou základní datové typy? (Dframe, series, časové řady)
    - základní orientace, indexování,

## Druhá lekce, kaggle
* reading and writting DFs,
* indexing, selecting rows & columns, conditional selection,
* functions, map and lambdas,
* groupby, multiindex, sorting values,
* data types or Dtypes, convert data types,
* missing values, fillna, replace,
* rename, merge, join, concat.
* missing data points, the reason why are the data missing, drop missing values, filling missing vals,
* scaling: changing the range, normalization: changing shape,
* parsing dates, selecting days,
* encoding: what are encodings, encode, decode, identifying and saving examples,
* inconsistency: fuzzywuzzy

## Druhá lekce, kniha
* pandas objects: DFs, Series, Index,
* indexing and selection: series, DFs,
* operating: w/ numpy (hard to implement without numpy),
* handling missing data: Nan and None in pandas,detecting, droping, filling null values,
* indexing: single index, multiple indexes(bad and good example), indexing and slicing multiindex,
* concat, append, merge, join,
* agregation: count, first, last, mean, median, min, max,
* groupBy: split, apply, combine,
* pivot table, methods similar to python methods, regex finding,slice & other miscellaneous methods,
* Time series, to_datetime, data structures of time series, date_range, period_range, timedelta_range,
* frenquencies & offsets, resampling, time shifting,
* eval and query, comparing eval and without eval,


## Druhá lekce, Honza
* manipulace, úvod, instalace,
* DataFrame,
* Series,
* Načítání dat,
* selekce dat,
* indexování, loc a iloc,
* přejmenování, přidávání, odstraňování sloupců,
* Nan, null, nahrazování,
* concat, merge, drop_duplicates, sort_values,
* unique, apply,
* kopie a práce s kopiemi,
* statistika,
* čtení dat online,

* [High performance](),
    - [úvodní motivace](),
    - [eval](),
    - [query](),
    - [caveats]().

## funkce EVAL

Funkce eval() v knihovně pandas slouží k efektivnímu vyhodnocení aritmetických výrazů na pandas DataFrame nebo Series objektech. Její největší výhodou je rychlost a efektivita, která spočívá v minimalizaci paměťové zátěže a zrychlení operací.

Když pracujete s velkými datovými sadami, paměťová zátěž a rychlost se stávají klíčovými faktory. Funkce eval() optimalizuje výpočetní proces tím, že snižuje množství alokované paměti a zrychluje výpočetní operace.

Ukázka použití funkce eval() v pandas:

In [ ]:
import pandas as pd

# Vytvoření vzorového DataFrame
data = {'A': [1, 2, 3, 4, 5],
        'B': [6, 7, 8, 9, 10],
        'C': [11, 12, 13, 14, 15]}

df = pd.DataFrame(data)

# Výpočet nového sloupce 'D' pomocí funkce eval()
df['D'] = df.eval('A + B * C')
print(df)


In [ ]:
Alternativní způsob, jak provést stejný výpočet bez použití funkce eval(), je následující:

In [ ]:
df['D'] = df['A'] + df['B'] * df['C']


In [ ]:
Avšak v případě velkých datových sad nebo složitějších výrazů může být použití funkce eval() výhodnější z hlediska rychlosti a paměťové efektivity.